In [ ]:
import numpy as np
import re
from scipy.spatial import KDTree
import glob
import math
import os.path as path
import os

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def parse_stream_file_v2(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    header = []
    chunks = []
    chunk_text = []
    in_header = True  # Flag to separate header from chunks
    tan_squared_10_degrees = math.tan(math.radians(10)) ** 2

    for line in lines:
        linestrip = line.strip()
        
        # Collect header lines until the first chunk starts
        if in_header:
            if linestrip == '----- Begin chunk -----':
                in_header = False
                chunk_text = [line]  # Start a new chunk
            else:
                header.append(line)  # Add to header
        else:
            if linestrip == '----- Begin chunk -----':
                chunk_text = [line]  # Start a new chunk
            elif linestrip == '----- End chunk -----':
                chunk_text.append(line)  # End the current chunk
            elif linestrip.startswith('cstar ='):
                chunk_text.append(line) 
                parts = linestrip.split()
                x, y, z = float(parts[2]), float(parts[3]), float(parts[4])
                # If it meets the condition, add it to the chunks list
                if z**2 / (x**2 + y**2) >= tan_squared_10_degrees:
                    chunks.append({'text': chunk_text})
            else:
                chunk_text.append(line)  # Continue building the current chunk

    return header, chunks

def write_chunks_to_file(header, chunks, output_file_path):
    with open(output_file_path, 'w') as outfile:
        # Write the header first
        for line in header:
            outfile.write(line)
        # Write each chunk that meets the condition
        for chunk in chunks:
            for line in chunk['text']:
                outfile.write(line)



folder_path = "/path/to/your/stream/file"
streams = [s for s in os.listdir(folder_path) if s.endswith('.stream') and not s.startswith('header')]

for s in streams: 
    s_name = s.rpartition('.')[0]
    print(f"working on file {s}")
    input_file_path = path.join(folder_path, s)
    output_file_path = path.join(folder_path, f"{s_name}_c-filtered.stream")
    
    # Parse each stream file for header and chunks
    header, chunks = parse_stream_file_v2(input_file_path)
    
    # Write the header and filtered chunks to the new file
    write_chunks_to_file(header, chunks, output_file_path)
